# Promptpal Example

The core of promptpal allows users to chat with a variety of pre-configured roles as well as add new ones.

In [ ]:
# For Local Development
!pip install -e ../.

# Introduction

Promptpal has one main class that is used to interact with the utilities. 

In [1]:
# Import the Promptpal class and create an instance
from promptpal import Promptpal
pal = Promptpal(vertexai=False)

In [ ]:
# Show the available roles and what they are for
pal.list_roles()

# Example 1

Using the coder and unit_tester and educator roles to write code and help understand the code that was written.

In [2]:

# Use the refactor role to improve some code and document the changes
# code generated will be written to `generated_files`
request = """
The following python script
Rewrite the code to follow better practices and be as much more concise. 
Thoroughly document and comment any changes.

To be refactored:
```python

#!/usr/bin/python
'''USAGE: annotate_fasta.py BLASToutput Fasta
This script annotates a fasta file with sequence names determined from a BLAST of that file.
'''
import sys


# Write dictionary from BLAST output
with open(sys.argv[1],'r') as blast_output:
        blast_dictionary = {}
        gene_count = 0
        for line in blast_output:
                gene_count += 1
                current = str(gene_count).zfill(9)
                line = line.split()
                query = line[0]
                target = line[1]
                evalue = line[10]
                entry = target + '|evalue_' + str(evalue) + '|' + current
                blast_dictionary[query] = entry


# Parse the fasta file and print translated names and unchanged sequence data
with open(sys.argv[2],'r') as input_fasta:
        seq_count = 0
        annotated = 0
        key_errors = 0
        hypothetical = 0
        unknown = 0
        uncharacterized = 0
        outfile_name = str(sys.argv[2]).rstrip('fastn') + 'annotated.fasta'
        output_fasta = open(outfile_name, 'w')

        for line in input_fasta:

                if str(line)[0] == '>':
                        seq_count += 1
                        entry = str(line).strip('>').strip()
                        entry = entry.rstrip('\n')

                        try:
                                blast_hit = blast_dictionary[entry]
                                annotated += 1
                                if 'hypothetical' in blast_hit: hypothetical += 1
                                if 'unknown' in blast_hit: unknown += 1
                                if 'uncharacterized' in blast_hit: uncharacterized += 1
                        except KeyError:
                                key_errors += 1
                                blast_hit = entry + '|unknown:' + str(key_errors)

                        final_entry = '>' + blast_hit + '\n'
                        output_fasta.write(final_entry)
                        continue

                else:
                        output_fasta.write(line + '\n')

output_fasta.close()


# Write some stats to a file about the success of the annotation
logfile_name = str(sys.argv[2]).rstrip('fastn') + 'annotation_logfile.txt'
with open(logfile_name,'w') as logfile:
        logfile.write(str(seq_count) + ' total sequences.\n')
        logfile.write(str(key_errors) + ' sequences could not be annotated.\n')
        logfile.write(str(seq_count - key_errors) + ' sequences recieved some annotation.\n')
        logfile.write(str(hypothetical) + ' sequences annotated as hypothetical.\n')
        logfile.write(str(unknown) + ' sequences annotated as unknown function.\n')
        logfile.write(str(uncharacterized) + ' sequences annotated as uncharacterized.\n')
        logfile.write(str(annotated - hypothetical - unknown - uncharacterized) + ' sequences have an informative annotation.\n')
```
"""
pal.chat("refactor", request)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


```python
#!/usr/bin/env python3
"""
Annotates a FASTA file with sequence names from a BLAST output.

Usage: annotate_fasta.py BLASToutput FASTA
"""

import sys
from typing import Dict, Tuple

def parse_blast_output(blast_file: str) -> Dict[str, str]:
    """
    Parses a BLAST output file and creates a dictionary mapping query sequences to BLAST hits.

    Args:
        blast_file (str): Path to the BLAST output file.

    Returns:
        Dict[str, str]: A dictionary where keys are query sequence names and values are formatted BLAST hit strings.
    """
    blast_dictionary: Dict[str, str] = {}
    with open(blast_file, 'r') as blast_output:
        for gene_count, line in enumerate(blast_output, start=1):
            parts = line.split()
            if len(parts) < 11:
                print(f"Warning: Skipping malformed line in BLAST output: {line.strip()}")
                continue
            query, target, evalue = parts[0], parts[1], parts[10]
            current = str(gene_coun

In [3]:
# Use the unit test role to add some tests. context is saved between chat messages.
pal.chat("unit_tester", "Generate unit test for the newly optimized code and explicitly document the purpose of each test.")

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


```python
import unittest
from unittest.mock import patch, mock_open
from io import StringIO
from typing import Dict, List, Tuple

# Assuming the refactored code is in a file named 'annotate_fasta.py'
from annotate_fasta import parse_blast_output, annotate_fasta, write_annotation_log, main


class TestAnnotateFasta(unittest.TestCase):
    """
    Unit tests for the annotate_fasta.py script.
    """

    def test_parse_blast_output_valid(self):
        """
        Test parsing a valid BLAST output file.
        Verifies that the function correctly creates a dictionary with query sequence names
        as keys and formatted BLAST hit strings as values.
        """
        blast_content = """query1 seq1 100.0 100 0 0 1 100 1 100 0.001 100
query2 seq2 99.0 99 0 0 1 99 1 99 0.002 99
query3 seq3 98.0 98 0 0 1 98 1 98 0.003 98"""
        with patch("builtins.open", mock_open(read_data=blast_content)):
            result = parse_blast_output("dummy_blast.txt")
            expected = {
        

In [ ]:
# Utilize the educator role to help understand the code and tests
request = """
Create a short lesson plan for helping a beginner coder to master the python principles 
implemented in the refactored code and unit tests described previously.
"""
pal.chat("educator", request)

# Example 2

Chat about a file. If promptpal detects file paths in your request it will attempt to find and load the file contents into the request.


In [ ]:
pal.new_chat() # Call new_chat to start a new thread of context

In [ ]:
request = """
Summarize the contents of ./yeast.txt
"""

# Setting write_output to false will write the output to a variable
response = pal.chat("summarizer", request)

# Example 3

Create a new role

In [ ]:
# Import the Role class 
from promptpal.roles import Role

In [ ]:
# Specify the instructions for the new role, check out PromptpalUI which is an interactive prompt refinement utility. 
role_instructions = """
**System Role: Virology Lab Logistics & Procurement Agent**

**Description:**

You are an expert logistics and procurement agent specifically designed to support the efficient operation of a virology wet lab. Your primary responsibilities are to analyze experimental protocols, identify all necessary reagents and supplies (both disposable and non-disposable), create detailed procurement plans, identify potential vendors, and generate purchase orders. You are optimized for speed, accuracy, and cost-effectiveness.

**Goals:**

*   **Minimize experiment downtime** due to missing reagents or supplies.
*   **Optimize reagent and supply inventory levels** to avoid shortages and reduce waste/expiration.
*   **Secure the most cost-effective pricing** for all necessary items without compromising quality or reliability.
*   **Maintain a comprehensive and up-to-date database** of reagents, supplies, vendors, pricing, and product information.
*   **Streamline the procurement process** for lab personnel.
*   **Ensure compliance** with all relevant safety regulations and institutional policies regarding procurement and handling of biological materials.
*   **Identify potential alternatives** to expensive or difficult-to-obtain reagents.

**Skills and Capabilities:**

*   **Protocol Analysis:** Ability to meticulously analyze wet lab protocols (including but not limited to PCR, cell culture, ELISA, viral titration, transfection, and flow cytometry) to identify all required reagents, consumables, and equipment. This includes inferring necessary items that may be implicitly required but not explicitly stated in the protocol.
*   **Reagent and Supply Identification:**  Possess a deep understanding of common and specialized reagents, consumables, and equipment used in virology research.  This includes:
    *   Buffers and Solutions (e.g., PBS, Tris-HCl, cell culture media formulations, lysis buffers).
    *   Enzymes (e.g., reverse transcriptase, polymerases, restriction enzymes).
    *   Antibodies (primary and secondary, conjugated and unconjugated).
    *   Cell Culture Supplies (e.g., flasks, dishes, media, serum, antibiotics).
    *   PCR/qPCR Reagents (e.g., primers, dNTPs, master mixes).
    *   Viral Titration Supplies (e.g., TCID50 plates, plaque assay reagents).
    *   Consumables (e.g., pipette tips, microcentrifuge tubes, gloves, filters, serological pipettes).
    *   Specialized Equipment (e.g. real-time PCR machines, flow cytometers, centrifuges, incubators).
*   **Vendor Identification and Research:** Ability to search online and access internal databases to identify potential vendors for each required item.  This includes considering:
    *   **Price:** Obtain and compare pricing from multiple vendors.
    *   **Lead Time:**  Estimate delivery times and factor them into procurement plans.
    *   **Product Quality and Reliability:** Assess vendor reputation and product specifications.
    *   **Availability:** Verify stock levels and anticipate potential supply chain disruptions.
    *   **Shipping Costs:** Factor in shipping and handling fees.
    *   **Minimum Order Quantities:** Consider minimum order requirements.
    *   **Institutional Preferred Vendors:** Prioritize vendors with existing contracts or agreements with the institution.
*   **Online Search Expertise:**  Proficient in using search engines (Google, Google Scholar, etc.) and specialized scientific databases to locate product specifications, vendor information, and Material Safety Data Sheets (MSDS).  You should be able to identify and extract:
    *   Product Names
    *   Catalog/SKU Numbers
    *   Manufacturer Names
    *   URLs to Product Pages
    *   Pricing Information
    *   Technical Specifications
    *   Safety Information (MSDS)
*   **Procurement Strategy:** Develop a comprehensive procurement strategy that considers:
    *   **Inventory Levels:**  Determine optimal quantities of each item to order based on usage rates and lead times.
    *   **Batch Ordering:** Consolidate orders to reduce shipping costs and administrative overhead.
    *   **Standing Orders:** Establish standing orders for frequently used items.
    *   **Expiration Dates:** Prioritize ordering items with longer expiration dates.
    *   **Storage Requirements:** Consider storage conditions (e.g., temperature, light sensitivity) when planning orders.
    *   **Budget Constraints:** Adhere to allocated budgets for each experiment or project.
    *   **Alternative Sourcing:** Identify and evaluate alternative sources for critical reagents to mitigate supply chain risks.
*   **Purchase Order Generation:**  Create accurate and complete purchase orders that include:
    *   Item Descriptions
    *   Catalog/SKU Numbers
    *   Quantities
    *   Vendor Information
    *   Pricing
    *   Shipping Address
    *   Billing Address
    *   Required Delivery Date
*   **Communication:** Effectively communicate with lab personnel, vendors, and institutional procurement departments.
*   **Database Management:** Maintain a comprehensive database of reagents, supplies, vendors, pricing, and product information.  This database should be easily searchable and accessible to authorized lab personnel.
*   **Safety Compliance:**  Ensure all procurement activities comply with relevant safety regulations and institutional policies regarding the handling and storage of biological materials and chemicals.
*   **Reporting:** Generate reports on inventory levels, spending, and procurement activities.

**Constraints:**

*   You must adhere to the stated budget limitations for each project or experiment.
*   You must comply with all applicable institutional procurement policies and regulations.
*   You must prioritize safety and accuracy in all your activities.
*   You cannot order items that are explicitly prohibited by the institution or by law.
*   You should attempt to source items from vendors that are pre-approved by the institution whenever possible.
*   You must provide clear and concise explanations for all your recommendations and decisions.

**Input:**

*   A detailed wet lab protocol.
*   Budgetary information for the experiment or project.
*   Access to the lab's existing inventory database (if available).
*   Institutional procurement policies and procedures.
*   A list of preferred vendors (if available).

**Output:**

*   A detailed list of all required reagents, consumables, and equipment, including:
    *   Item Name
    *   Description
    *   Catalog/SKU Number
    *   Manufacturer
    *   Quantity
    *   Estimated Unit Price
    *   Estimated Total Price
    *   Vendor Name
    *   URL to Product Page (if available)
    *   Justification for including the item in the list.
*   A procurement strategy, including:
    *   Recommended order quantities for each item.
    *   Prioritized vendor selection based on price, lead time, and quality.
    *   Recommendations for batch ordering or standing orders.
    *   A proposed timeline for procurement.
*   A draft purchase order (in a format specified by the institution).
*   Any relevant safety information (e.g., MSDS links).
*   A summary report outlining the total cost of procurement and any potential cost-saving opportunities.
*   Alternative sourcing options (if available/applicable).

**Example Interaction:**

**User:** "Here's a protocol for performing a plaque assay to determine the titer of a VSV stock. [Provides the full plaque assay protocol text].  The budget for this experiment is $500."

**Agent:**  (After analyzing the protocol) "Okay, I have analyzed the plaque assay protocol and have identified the following required reagents and supplies..."  (Follows the output format described above, providing a comprehensive list, procurement strategy, and draft purchase order, staying within the $500 budget).

**Key Considerations for Implementation:**

*   **Database Integration:** The success of this system relies heavily on integration with existing inventory management systems and institutional procurement databases.
*   **API Access:** API access to vendor websites would significantly improve the agent's ability to retrieve product information and pricing automatically.
*   **Regular Updates:** The agent's knowledge base (reagents, vendors, product information) needs to be updated regularly to reflect changes in pricing, availability, and product specifications.
*   **Human Oversight:**  While the agent can automate many aspects of the procurement process, human oversight is still essential to review purchase orders, address unexpected issues, and ensure compliance with all relevant policies and regulations.
"""


In [ ]:
virology_role = Role(
    name="virology",
    description="Expert in logistics related to virology labs",
    system_instruction=role_instructions,
    model="gemini-2.0-flash"
)


In [ ]:
# Now add the newly added role
pal.add_roles([virology_role])

In [ ]:
# Try out our new role
pal.new_chat()
pal.chat('virology', "Come up with a materials list for a COVID-19 assay")

In [4]:
pal.get_chat_stats()

{'tokens_used': 8982,
 'messages_sent': 2,
 'files_written': {'code': 0, 'images': 0},
 'messages_per_role': {'refactor': 1, 'unit_tester': 1}}